In [1]:
import sys, os
# sys.path.insert(0, '/content/drive/MyDrive/CloudsDL/BigEarthNet_exe_data/bigearthnet-models-tf-master')
# from BigEarthNet import BigEarthNet

import numpy as np
import pandas as pd
import re

# import tensorflow.compat.v1 as tf
import subprocess, time, os
import argparse
import json
import glob
import importlib
from osgeo import gdal, osr
gdal.UseExceptions()
from concurrent.futures import ThreadPoolExecutor
import copy
from pathlib import Path
import geopandas as gpd
import shapely
from pprint import pprint
from PIL import Image

import cv2
from tqdm import tqdm
from skimage.util.shape import view_as_windows

import tensorflow as tf
from tensorflow import keras
import pandas as pd

2023-01-13 11:16:47.801177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-13 11:16:47.961422: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-13 11:16:47.961460: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-13 11:16:49.923639: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
pwd

'E:\\Users\\sentinel_industry'

In [33]:
tiles = pd.read_csv('E://Users//sentinel_industry//deploy/but_nanjing', index_col=0)

In [34]:
scene_ids = tiles.index.to_numpy()

In [8]:
# scene_ids = nl#['S2A_35VLF_20180510_0_L2A', 'S2B_35VLF_20220524_0_L2A']

output_layers = '/gws/nopw/j04/aopp/manshausen/deploy/deploy_outl/'

image_download_folder = '/gws/nopw/j04/aopp/manshausen/deploy/deploy_dwnimg/'
out_put_blocks = '/gws/nopw/j04/aopp/manshausen/deploy/deploy_blk'

In [36]:

os.environ['GS_NO_SIGN_REQUEST']='YES'
os.environ['AWS_NO_SIGN_REQUEST']='YES'
os.environ['AWS_VIRTUAL_HOSTING'] ='FALSE'
os.environ['AWS_HTTPS'] = 'YES'
os.environ['CPL_VSIL_CURL_ALLOWED_EXTENSIONS'] = ".tif,.tiff,.jp2,.mrf,.idx,.lrc,.mrf.aux.xml,.vrt"
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'YES'


In [7]:

def get_asw_s2_path(scid):
    http_prefix = '/vsicurl/http://sentinel-cogs.s3.amazonaws.com/sentinel-s2-l2a-cogs/'
    
    http_path = http_prefix+\
                re.findall(r'\d+',scid.split('_')[1])[0]+'/'+\
                re.findall("[a-zA-Z]+", scid.split('_')[1])[0][0]+'/'+\
                re.findall("[a-zA-Z]+", scid.split('_')[1])[0][1:3]+'/'+\
                scid[10:14]+'/'+\
                str(int(scid[14:16]))+'/'+\
                scid+'/' 
    return http_path

def download_worker_bands(payload):
        
        payload['download_status'] = []
        try:
            if payload['ep'] == 'aws_cog':
                
                path=payload["aws_path"]+payload['download_band']+'.tif'
                now_ras = gdal.Open(path)
                gdal.GetDriverByName('GTiff')\
                    .CreateCopy(str(Path(payload['outfolder'], 
                                        payload['id']+"_"+payload['download_band']+'.tif')), now_ras)
                now_ras=None
                payload['download_status'] = payload['download_status']+['from aws to : '
                                        +str(Path(payload['outfolder'], 
                                        payload['id']+"_"+payload['download_band']+'.tif'))]
        except RuntimeError as err:
            print('Dang... ')

def download_worker(payload):
    task_list = []
    for band in payload['download_bands']:
        payload['download_band'] = band
        task_list.append(copy.deepcopy(payload))
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(download_worker_bands, task_list)


def download_df(scene_df, output_folder, bands_l2a = None, 
                                        end_point='aws_cog'):
        # Download for AWS
        if end_point == 'aws_cog':

            bands = [band for band in bands_l2a if bands_l2a[band]]
            #download_df['download_bands'] = bands
            scene_df['outfolder'] = output_folder
            scene_df['ep'] = end_point
            task_list = list(scene_df.T.to_dict().values())

            for tsk in task_list:
                tsk['download_bands'] =bands 
            with ThreadPoolExecutor(max_workers=4) as executor:
                tqdm(executor.map(download_worker, task_list), total=len(task_list))
            #scene_df = pd.DataFrame(task_list)

def output_image(output_img, nowGt, ImPrj, arr, data_type):

        nowBlockDs = gdal.GetDriverByName('GTiff').Create(output_img, 
                                                            arr.shape[2], 
                                                            arr.shape[1],
                                                            arr.shape[0],
                                                            data_type)
                                                            #gdal.GDT_UInt16)
        nowBlockDs.SetGeoTransform(nowGt)
        nowBlockDs.SetProjection(ImPrj)
        
        for i in range(arr.shape[0]):
            nowBlockBand = nowBlockDs.GetRasterBand(i+1)
            nowBlockBand.WriteArray(arr[i])
            nowBlockBand.SetNoDataValue(0)
            nowBlockBand.FlushCache()
        nowBlockDs=None

def bounds_to_polygon(bounds):
    '''
    function that converts a bounds list to a shapely Polygon object
    bounds :: list :: [xmin, ymin, xmax, ymax]
    return :: Shapely Polygon object
    '''
    return shapely.geometry.Polygon([[bounds[0],bounds[3]],
                    [bounds[0],bounds[1]],
                    [bounds[2],bounds[1]],
                    [bounds[2],bounds[3]]])

'deploy/deploy_dwnimg/S2A_50RNP_20210421_0_L2A_B02.tif'

In [14]:
os.path.isfile('deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_B02.tif')

True

In [37]:
# Download images
df_list=[]
for scene in scene_ids:
    if not os.path.isfile(image_download_folder+scene+'_B02.tif'):
        df_list.append([scene, get_asw_s2_path(scene)])
    else: print('already done for: ', scene)
dwnld_df = pd.DataFrame(df_list, columns=['id','aws_path'])

platform_bands_l2a={"B02": True,"B03": True,
                    "B04": True,"B08":True}
output_folder=image_download_folder
download_df(dwnld_df, output_folder,  bands_l2a = platform_bands_l2a, end_point='aws_cog')

base_dir = image_download_folder


already done for:  S2A_51SUT_20210408_0_L2A
already done for:  S2A_51SUU_20210408_0_L2A
already done for:  S2B_50SNB_20210605_0_L2A
already done for:  S2A_50SMD_20210524_0_L2A
already done for:  S2A_50RNV_20210918_0_L2A
already done for:  S2B_50SMC_20210509_0_L2A
already done for:  S2B_50SNA_20210605_0_L2A
already done for:  S2A_50SPC_20210730_1_L2A
already done for:  S2A_50SMB_20210911_0_L2A
already done for:  S2B_50RQV_20210622_0_L2A
already done for:  S2A_50SMA_20210404_0_L2A
already done for:  S2A_50SPA_20210501_0_L2A
already done for:  S2A_50RPV_20210918_0_L2A
already done for:  S2B_51STR_20210622_0_L2A
already done for:  S2A_51STS_20210408_0_L2A
already done for:  S2A_51STT_20210408_0_L2A
already done for:  S2A_51SUS_20210408_0_L2A
already done for:  S2A_50SPD_20210521_0_L2A
already done for:  S2B_50SPB_20210605_0_L2A
already done for:  S2B_51SUR_20210622_0_L2A
already done for:  S2A_50SQD_20210521_0_L2A
already done for:  S2B_50RNT_20210625_0_L2A
already done for:  S2B_50RNU_202

In [23]:
# scene_ids = scene_ids[1:]

In [38]:
images_df_list = [[("_").join(x.split("_")[1:3]),
                    x.split("_")[-1].split('.')[0],
                    os.path.join(base_dir,x)] for x in os.listdir(base_dir) if x.endswith('.tif')]
images_df = pd.DataFrame(images_df_list, columns= ['id', 'band', 'path'])


In [39]:
images_df

,id,band,path
0,50RMT_20210928,B02,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
1,50RMT_20210928,B03,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
2,50RMT_20210928,B04,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
3,50RMT_20210928,B08,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
4,50RNV_20210918,B02,deploy/deploy_dwnimg/S2A_50RNV_20210918_0_L2A_...
...,...,...,...
139,51STU_20210523,B08,deploy/deploy_dwnimg/S2B_51STU_20210523_0_L2A_...
140,51SUR_20210622,B02,deploy/deploy_dwnimg/S2B_51SUR_20210622_0_L2A_...
141,51SUR_20210622,B03,deploy/deploy_dwnimg/S2B_51SUR_20210622_0_L2A_...
142,51SUR_20210622,B04,deploy/deploy_dwnimg/S2B_51SUR_20210622_0_L2A_...


In [42]:
####################################################################
# Optional
####################################################################
# Save RGB images for visuals
for scene in scene_ids:
    stbits = scene.split('_')
    shortname = stbits[1]+'_'+stbits[2]
    shortname
    if not os.path.isfile(os.path.join(output_layers,shortname+'_rgb.tif')):
        print('saving visuals for: ', scene)
      # red
        redDs = gdal.Open(os.path.join(image_download_folder,scene+'_B04.tif'))
        redArr = redDs.ReadAsArray()
        nowGt = redDs.GetGeoTransform()
        ImPrj = redDs.GetProjection()
        redDs=None
        rgb = redArr[np.newaxis,...]
        # green
        greenDs = gdal.Open(os.path.join(image_download_folder,scene+'_B03.tif'))
        greenArr = greenDs.ReadAsArray()
        greenDs=None
        rgb = np.append(rgb, greenArr[np.newaxis,...], axis=0)
        # blue
        blueDs = gdal.Open(os.path.join(image_download_folder,scene+'_B02.tif'))
        blueArr = blueDs.ReadAsArray()
        blueDs=None
        rgb = np.append(rgb, blueArr[np.newaxis,...], axis=0)
    #     if os.path.isdir(os.path.join(output_layers, scene)) == False:
    #         os.makedirs(os.path.join(output_layers, scene))
        
        output_image(os.path.join(output_layers,shortname+'_rgb.tif'), nowGt, ImPrj, rgb, gdal.GDT_UInt16)

    else: print('already done for: ', scene)

already done for:  S2B_51RUN_20210510_0_L2A


In [218]:
# stbits = scene.split('_')
# shortname = stbits[1]+'_'+stbits[2]
# shortname

'50SND_20210419'

In [170]:
# images_df = images_df[4:8]

In [211]:
images_df

,id,band,path
0,50RMT_20210928,B02,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
1,50RMT_20210928,B03,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
2,50RMT_20210928,B04,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
3,50RMT_20210928,B08,deploy/deploy_dwnimg/S2A_50RMT_20210928_0_L2A_...
4,50RNV_20210918,B02,deploy/deploy_dwnimg/S2A_50RNV_20210918_0_L2A_...
...,...,...,...
139,51STU_20210523,B08,deploy/deploy_dwnimg/S2B_51STU_20210523_0_L2A_...
140,51SUR_20210622,B02,deploy/deploy_dwnimg/S2B_51SUR_20210622_0_L2A_...
141,51SUR_20210622,B03,deploy/deploy_dwnimg/S2B_51SUR_20210622_0_L2A_...
142,51SUR_20210622,B04,deploy/deploy_dwnimg/S2B_51SUR_20210622_0_L2A_...


In [44]:
####################################################################
# BLOCKS
####################################################################


band_order = ["B02","B03","B04","B08",]
s2_dims = {10: 10980, 20: 5490, 60: 1830}

in_shape={'B01': 20,
          'B02': 120,
          'B03': 120,
          'B04': 120,
          'B05': 60,
          'B06': 60,
          'B07': 60,
          'B08': 120,
          'B09': 20,
          'B11': 60,
          'B12': 60,
          'B8A': 60,}

uni_scenes = images_df['id'].unique()
for uscn in uni_scenes:
    print('For: ', uscn)
    if os.path.isdir(os.path.join(out_put_blocks, uscn)) == True:
        print('already blocked for: ', uscn)
        continue
    else:
        os.makedirs(os.path.join(out_put_blocks, uscn))
    output_path = os.path.join(out_put_blocks, uscn)

    # Stack Bands
    #------------
    now_images = images_df.loc[images_df['id']==uscn]
    for band in band_order:
        try:
#             print(now_images.loc[now_images['band']==band]['path'].iloc[0])
            nowDs = gdal.Open(now_images.loc[now_images['band']==band]['path'].iloc[0])
            gt= nowDs.GetGeoTransform()
            proj= nowDs.GetProjection()
            windows=nowDs.ReadAsArray()
            windows=windows[np.newaxis,...]


            # Define tilting parameters
            # -------------------------
            rasXmax = nowDs.RasterXSize
            rasYmax = nowDs.RasterYSize
            block_size = 230       # Size of tile in pixels (X size Y size)

            overlap_perc = 0                    # Overlap of tiles

            overlap = 90#int(round(block_size*(overlap_perc/100),0))
            stepSize = block_size-overlap

            #print('Blocking ...')
            vrt_list = []
            #------------------------------------
            # Manage edge case
            #------------------------------------
            calc_x_steps = (rasXmax-overlap)/stepSize
            calc_y_steps = (rasYmax-overlap)/stepSize
            last_square =False
            if calc_y_steps != int(calc_y_steps):
                yminLim=0
                ymaxLim=block_size
                for y_step in range(int(calc_y_steps)):
                    nowBlockArr = windows[...,yminLim:ymaxLim,rasXmax-block_size:rasXmax]

                    yminLim = yminLim+stepSize
                    ymaxLim = ymaxLim+stepSize

                    nowGt = (gt[0] + ((rasXmax-block_size)*gt[1]), 
                              gt[1],0,
                              gt[3] - (int(y_step)*abs(stepSize*gt[5])),
                              0,gt[5])

                    out_file = os.path.join(output_path, uscn+'_'+band+'_BEN_'+str(y_step)+'_'+str(int(calc_x_steps)+1)+'.tif')
                    output_image(out_file,
                              nowGt, proj, nowBlockArr, gdal.GDT_UInt16)
                last_square=True

            if calc_x_steps !=int(calc_x_steps):
              # edge case - calculate additional blocks for y max
                xminLim=0
                xmaxLim=block_size
                for x_step in range(int(calc_x_steps)):

                    nowBlockArr = windows[..., rasYmax-block_size:rasYmax,xminLim:xmaxLim]
                    xminLim = xminLim+stepSize
                    xmaxLim = xmaxLim+stepSize

                    nowGt = (gt[0] + (int(x_step)*abs(stepSize*gt[1])), 
                              gt[1],0,
                              gt[3] - ((rasYmax-block_size)*abs(gt[5])),
                              0,gt[5])

                    out_file = os.path.join(output_path, uscn+'_'+band+'_BEN_'+str((int(calc_y_steps)+1))+'_'+str(x_step)+'.tif')
                    output_image(out_file,
                              nowGt, proj, nowBlockArr, gdal.GDT_UInt16)
                last_square=True

            #------------------------------------
            # Get Last square
            #------------------------------------
            if last_square:
                nowBlockArr = windows[..., rasYmax-block_size:rasYmax,rasXmax-block_size:rasXmax]

                nowGt = (gt[0] + ((rasXmax-block_size)*gt[1]), 
                          gt[1],0,
                          gt[3] - ((rasYmax-block_size)*abs(gt[5])),
                          0,gt[5])

                out_file = os.path.join(output_path, uscn+'_'+band+'_BEN_'+str((int(calc_y_steps)+1))+'_'+str((int(calc_x_steps)+1))+'.tif')
                output_image(out_file,
                          nowGt, proj, nowBlockArr, gdal.GDT_UInt16)

            #------------------------------------
            # Continue normal windows
            #------------------------------------
            windows = view_as_windows(windows,(windows.shape[0],block_size, block_size), step=stepSize)
            windows = windows[0]

            y_size = windows.shape[0]
            x_size = windows.shape[1]
            for y in tqdm(range(y_size)):
                for x in range(x_size):
                    y=y # For testing
                    x=x
                    nowBlockArr = windows[y][x]

                    nowGt = (gt[0] + (x*abs(stepSize*gt[1])), 
                              gt[1],0,
                              gt[3] - (y*abs(stepSize*gt[5])),
                              0,gt[5])    

                    out_file = os.path.join(output_path, uscn+'_'+band+'_BEN_'+str(y)+'_'+str(x)+'.tif')

                    output_image(out_file,
                              nowGt, proj, nowBlockArr, gdal.GDT_UInt16)
        except ValueError:
            print('Failed: ', now_images.loc[now_images['band']==band]['path'].iloc[0])
            break
#     break 

For:  50RMT_20210928
already blocked for:  50RMT_20210928
For:  50RNV_20210918
already blocked for:  50RNV_20210918
For:  50RPU_20210501
already blocked for:  50RPU_20210501
For:  50RPV_20210918
already blocked for:  50RPV_20210918
For:  50RQT_20210925
already blocked for:  50RQT_20210925
For:  50SMA_20210404
already blocked for:  50SMA_20210404
For:  50SMB_20210911
already blocked for:  50SMB_20210911
For:  50SMD_20210524
already blocked for:  50SMD_20210524
For:  50SPA_20210501
already blocked for:  50SPA_20210501
For:  50SPC_20210730
already blocked for:  50SPC_20210730
For:  50SPD_20210521
already blocked for:  50SPD_20210521
For:  50SQD_20210521
already blocked for:  50SQD_20210521
For:  51RUQ_20210508
already blocked for:  51RUQ_20210508
For:  51STS_20210408
already blocked for:  51STS_20210408
For:  51STT_20210408
already blocked for:  51STT_20210408
For:  51SUS_20210408
already blocked for:  51SUS_20210408
For:  51SUT_20210408
already blocked for:  51SUT_20210408
For:  51SUU_20

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [00:31<00:00,  2.46it/s]

For:  50RNU_20210625
already blocked for:  50RNU_20210625
For:  50RQV_20210622
already blocked for:  50RQV_20210622
For:  50SMC_20210509
already blocked for:  50SMC_20210509
For:  50SNA_20210605
already blocked for:  50SNA_20210605
For:  50SNB_20210605
already blocked for:  50SNB_20210605
For:  50SNC_20210605
already blocked for:  50SNC_20210605
For:  50SND_20210419
already blocked for:  50SND_20210419
For:  50SPB_20210605
already blocked for:  50SPB_20210605
For:  50SQA_20210622
already blocked for:  50SQA_20210622
For:  50SQC_20210406
already blocked for:  50SQC_20210406
For:  51RTN_20210523
already blocked for:  51RTN_20210523
For:  51RTP_20210513
already blocked for:  51RTP_20210513
For:  51RTQ_20210523
already blocked for:  51RTQ_20210523
For:  51RUN_20210510
already blocked for:  51RUN_20210510
For:  51STR_20210622
already blocked for:  51STR_20210622
For:  51STU_20210523
already blocked for:  51STU_20210523
For:  51SUR_20210622
already blocked for:  51SUR_20210622


In [88]:
lmodel = keras.models.load_model('saved_models/resnet50v2/')

In [ ]:
###########################################
#STACK HERE
###########################################
'''
# For the scene - loop through unique grid ids (uni_grids)

for uni_gird in uni_grids:

# Red
redDs = gdal.Open(os.path.join(output_path, reconstruct_name+ (B02) +uni_gird+ '.tif'))
ImGT = redDs.GetGeoTransform()
ImPrj = redDs.GetProjection()
red_band = redDs.GetRasterBand(1)
red_arr = red_band.ReadAsArray()
redDs=None

#Green
greenDs = gdal.Open(os.path.join(output_path, reconstruct_name+ (B03) +uni_gird+ '.tif'))
green_arr = greenDs.ReadAsArray()
greenDs = None

#NIR
nirDs = gdal.Open(os.path.join(output_path, reconstruct_name+ (B04) +uni_gird+ '.tif'))
nir_arr = nirDs.ReadAsArray()
nirDs=None


nirDs = gdal.Open(os.path.join(output_path, reconstruct_name+ (B08) +uni_gird+ '.tif'))
nir_arr = nirDs.ReadAsArray()
nirDs=None

image_stack = np.stack((red_arr,green_arr,nir_arr))
output_img = /path/and/name
output_image(output_img, ImGT, ImPrj, image_stack, gdal.GDT_UInt16)
'''
ben_classes = ['coal', 'steel', 'other']

base_folder = 'deploy/deploy_blk/' 
for uni_scene in uni_scenes:
    if os.path.isfile(os.path.join(output_layers,'S2A_'+uni_scene + '_0_L2A',
                        uni_scene + '_BEN.gpkg')) or os.path.isfile(os.path.join(output_layers,
                        uni_scene + '_BEN.gpkg')) : 
        print('already done for ', uni_scene)
        continue
    
    print('For: ', uni_scene)

    bands_blocks = os.listdir(os.path.join(base_folder,uni_scene))
    bands_blocks_df = [[('_').join(x.split('_')[0:2]),
                        x.split('_')[2],
                        ('_').join(x.split('_')[-2:]),
                        os.path.join(base_folder, uni_scene, x)] for x in bands_blocks]
    bands_blocks_df = pd.DataFrame(bands_blocks_df, columns=['scene', 'band', 'block', 'path'])

    blocks=bands_blocks_df['block'].unique()
#     print(tqdm(blocks))
    probabilities_df=[]
    for block in blocks: 
        now_block=[]

        now_block.append(block[:-4])
        band_dict={}
        for band in band_order:
            nowBand_path = bands_blocks_df.loc[((bands_blocks_df['block'] == block) &\
                                                (bands_blocks_df['band'] == band))]['path'].iloc[0]
            nowBand = gdal.Open(nowBand_path)
            now_arr= nowBand.ReadAsArray()
            band_dict[band]=now_arr[np.newaxis,...]
#             print(band_dict.keys())
#             band_dict['Placeholder:0'] = np.array(False)
#             band_dict['BigEarthNet-19_labels_multi_hot']=np.expand_dims(np.zeros(19), 0)

            if band=='B04':
                geo_t = nowBand.GetGeoTransform()
                proj_now = nowBand.GetProjection()
                proj = osr.SpatialReference(wkt=proj_now)
                EPSG = int(proj.GetAttrValue('AUTHORITY',1))
                x_size = nowBand.RasterXSize
                y_size = nowBand.RasterYSize
                xmin = min(geo_t[0], geo_t[0] + x_size * geo_t[1])
                xmax = max(geo_t[0], geo_t[0] + x_size * geo_t[1])
                ymin = min(geo_t[3], geo_t[3] + y_size * geo_t[5])
                ymax = max(geo_t[3], geo_t[3] + y_size * geo_t[5])
            nowBand=None
        B02, B03, B04, B08 = band_dict['B02'][0], band_dict['B03'][0], band_dict['B04'][0], band_dict['B08'][0]
        B02 = (B02-ch1_mean)/ch1_std
        B03 = (B03-ch2_mean)/ch2_std
        B04 = (B04-ch3_mean)/ch3_std
        B08 = (B08-ch4_mean)/ch4_std
        features = np.array([ B02, B03, B04, B08]).transpose(1,2,0)
        probs = tf.nn.softmax(lmodel.predict(np.expand_dims(features,axis=0), verbose=0))
#       TF 2 MODEL PREDICT
#         probs= sess.run(model.probabilities, feed_dict=model.feed_dict(band_dict))
        for prob in probs[0]:
            now_block.append(prob.numpy())
        now_block.append(bounds_to_polygon([xmin, ymin, xmax, ymax]))
        probabilities_df.append(now_block)
    
    probabilities_df = gpd.GeoDataFrame(probabilities_df, columns=['block_id']+ben_classes+['geometry'])
    probabilities_df = probabilities_df.set_crs(EPSG, allow_override=True)
    probabilities_df.to_file(os.path.join(output_layers,
                        uni_scene + '_BEN.gpkg'), driver='GPKG')
    print('Done ', uni_scene + '_BEN.gpkg')
#     break 

already done for  50RMT_20210928
For:  50RNV_20210918


In [110]:
# for prob in pred[0]:
#     print(prob.numpy())

7.852733e-05
1.745435e-09
0.99992144


In [227]:
scene

'S2B_50SND_20210419_0_L2A'

In [229]:
'deploy/deploy_outl/'

'deploy/deploy_outl/'

In [233]:
finished_list = glob.glob(output_layers+'*/*_BEN.gpkg') + glob.glob(output_layers+'*_BEN.gpkg')

In [ ]:
for file in finished_list: 
    df = gpd.read_file(file)
    os.remove(file)

In [226]:
probabilities_df

,block_id,coal,steel,other,geometry
0,0_0,2.370391e-10,1.033139e-08,1.000000,"POLYGON ((300000.000 3600000.000, 300000.000 3..."
1,0_1,2.671659e-08,4.452243e-07,1.000000,"POLYGON ((301400.000 3600000.000, 301400.000 3..."
2,0_10,6.950425e-11,2.423769e-09,1.000000,"POLYGON ((314000.000 3600000.000, 314000.000 3..."
3,0_11,1.174801e-11,6.189567e-10,1.000000,"POLYGON ((315400.000 3600000.000, 315400.000 3..."
4,0_12,1.842797e-11,8.374229e-10,1.000000,"POLYGON ((316800.000 3600000.000, 316800.000 3..."
...,...,...,...,...,...
6079,9_75,3.293703e-04,7.909591e-07,0.999670,"POLYGON ((405000.000 3587400.000, 405000.000 3..."
6080,9_76,1.270806e-03,4.456614e-06,0.998725,"POLYGON ((406400.000 3587400.000, 406400.000 3..."
6081,9_78,5.058030e-04,1.866442e-05,0.999476,"POLYGON ((407500.000 3587400.000, 407500.000 3..."
6082,9_8,2.911574e-10,7.850241e-09,1.000000,"POLYGON ((311200.000 3587400.000, 311200.000 3..."


In [212]:
# probabilities_df = gpd.GeoDataFrame(probabilities_df, columns=['block_id']+ben_classes+['geometry'])
# probabilities_df = probabilities_df.set_crs(EPSG, allow_override=True)
# probabilities_df.to_file(os.path.join('E:/Users/sentinel_industry',output_layers,
#                         uni_scene + '_BEN.gpkg'), driver='GPKG')
# # probabilities_df.to_file('deploy/deploy_outl/S2A_50SPC_20210730_1_L2A/50SPC_20210730_BEN.shp')

C:\Anaconda3_64\envs\gdal\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [202]:
# p=os.path.join('E:/Users/sentinel_industry',output_layers,'S2A_'+uni_scene + '_0_L2A',
#                         uni_scene + '_BEN.shp')

In [235]:
# probabilities_df.sort_values('coal')

In [236]:
# probabilities_df.to_file(p)

In [237]:
# for uni_scene in uni_scenes:
#     print(os.path.isfile(os.path.join(output_layers,'S2A_'+uni_scene + '_0_L2A',
#                         uni_scene + '_BEN.gpkg')))

In [238]:
# uni_scenes

In [243]:
uni_scene

'51SUR_20210622'

In [254]:
ch1_mean, ch1_std = 970.4162, 1051.8454
ch2_mean, ch2_std = 1176.249, 1031.262
ch3_mean, ch3_std = 1273.2376, 1117.068
ch4_mean, ch4_std = 2266.9050, 1241.5509

for uni_scene in ['51STS_20210408']:
    
    bands_blocks = os.listdir(os.path.join(base_folder,uni_scene))
    bands_blocks_df = [[('_').join(x.split('_')[0:2]),
                        x.split('_')[2],
                        ('_').join(x.split('_')[-2:]),
                        os.path.join(base_folder, uni_scene, x)] for x in bands_blocks]
    bands_blocks_df = pd.DataFrame(bands_blocks_df, columns=['scene', 'band', 'block', 'path'])

    blocks=bands_blocks_df['block'].unique()
    probabilities_df=[]
    for block in ['16_65.tif']: 

        now_block=[]

        now_block.append(block[:-4])
        band_dict={}
        for band in band_order:
            nowBand_path = bands_blocks_df.loc[((bands_blocks_df['block'] == block) &\
                                                (bands_blocks_df['band'] == band))]['path'].iloc[0]
            nowBand = gdal.Open(nowBand_path)
            now_arr= nowBand.ReadAsArray()
            band_dict[band]=now_arr[np.newaxis,...]
#             print(band_dict.keys())
#             band_dict['Placeholder:0'] = np.array(False)
#             band_dict['BigEarthNet-19_labels_multi_hot']=np.expand_dims(np.zeros(19), 0)

            if band=='B04':
                geo_t = nowBand.GetGeoTransform()
                proj_now = nowBand.GetProjection()
                proj = osr.SpatialReference(wkt=proj_now)
                EPSG = int(proj.GetAttrValue('AUTHORITY',1))
                x_size = nowBand.RasterXSize
                y_size = nowBand.RasterYSize
                xmin = min(geo_t[0], geo_t[0] + x_size * geo_t[1])
                xmax = max(geo_t[0], geo_t[0] + x_size * geo_t[1])
                ymin = min(geo_t[3], geo_t[3] + y_size * geo_t[5])
                ymax = max(geo_t[3], geo_t[3] + y_size * geo_t[5])
            nowBand=None
        B02, B03, B04, B08 = band_dict['B02'][0], band_dict['B03'][0], band_dict['B04'][0], band_dict['B08'][0]
        B02 = (B02-ch1_mean)/ch1_std
        B03 = (B03-ch2_mean)/ch2_std
        B04 = (B04-ch3_mean)/ch3_std
        B08 = (B08-ch4_mean)/ch4_std
        features = np.array([ B02, B03, B04, B08]).transpose(1,2,0)
#         break
        probs = tf.nn.softmax(lmodel.predict(np.expand_dims(features,axis=0), verbose=0))

In [256]:
probs

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.7993669 , 0.14490439, 0.05572876]], dtype=float32)>

In [21]:
finished_list = glob.glob(output_layers+'*/*_BEN.gpkg') + glob.glob(output_layers+'*_BEN.gpkg')

In [22]:
finished_list

['/gws/nopw/j04/aopp/manshausen/deploy/deploy_outl/35VLF_20180510_BEN.gpkg',
 '/gws/nopw/j04/aopp/manshausen/deploy/deploy_outl/50RMT_20210928_BEN.gpkg']

In [23]:
for file in finished_list: 
    df = gpd.read_file(file)
#     os.remove(file)

/home/users/pete_nut/miniconda3/envs/sentinelind/lib/python3.8/site-packages/geopandas/geodataframe.py:643: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:
/home/users/pete_nut/miniconda3/envs/sentinelind/lib/python3.8/site-packages/geopandas/geodataframe.py:643: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [24]:
df.to_crs('WGS 84')

,block_id,coal,steel,other,geometry
0,0_0,3.292797e-09,2.439494e-11,1.000000,"POLYGON ((115.96458 29.82641, 115.96479 29.805..."
1,0_1,2.441139e-05,4.719058e-07,0.999975,"POLYGON ((115.97907 29.82652, 115.97928 29.805..."
2,0_10,4.039838e-06,4.187946e-08,0.999996,"POLYGON ((116.10946 29.82747, 116.10965 29.806..."
3,0_11,3.462207e-06,7.450064e-08,0.999996,"POLYGON ((116.12395 29.82756, 116.12413 29.806..."
4,0_12,2.558181e-06,2.960344e-07,0.999997,"POLYGON ((116.13844 29.82766, 116.13862 29.806..."
...,...,...,...,...,...
6079,9_75,1.486370e-08,3.773528e-07,1.000000,"POLYGON ((117.05128 29.71674, 117.05127 29.695..."
6080,9_76,1.169602e-08,1.965684e-07,1.000000,"POLYGON ((117.06576 29.71674, 117.06574 29.695..."
6081,9_78,1.898736e-10,2.254952e-09,1.000000,"POLYGON ((117.07713 29.71673, 117.07711 29.695..."
6082,9_8,1.314750e-06,2.746320e-08,0.999999,"POLYGON ((116.08153 29.71357, 116.08171 29.692..."


In [17]:
df.where(df.other<0.95)

,block_id,coal,steel,other,geometry
50,0_54,0.043544,0.506967,0.449489,"POLYGON ((375600.000 6600000.000, 375600.000 6..."
93,10_22,0.038186,0.633517,0.328297,"POLYGON ((330800.000 6586000.000, 330800.000 6..."
119,10_46,0.102410,0.737071,0.160519,"POLYGON ((364400.000 6586000.000, 364400.000 6..."
124,10_50,0.090541,0.711056,0.198403,"POLYGON ((370000.000 6586000.000, 370000.000 6..."
128,10_54,0.524363,0.226625,0.249011,"POLYGON ((375600.000 6586000.000, 375600.000 6..."
...,...,...,...,...,...
6044,9_43,0.000283,0.613406,0.386311,"POLYGON ((360200.000 6587400.000, 360200.000 6..."
6046,9_45,0.048139,0.091622,0.860240,"POLYGON ((363000.000 6587400.000, 363000.000 6..."
6050,9_49,0.034230,0.043238,0.922532,"POLYGON ((368600.000 6587400.000, 368600.000 6..."
6052,9_50,0.044858,0.723851,0.231291,"POLYGON ((370000.000 6587400.000, 370000.000 6..."


In [ ]:
['44QLG_20210418_BEN.gpkg',
'44QLH_20210403_BEN.gpkg',
'44QLJ_20210403_BEN.gpkg',
'44QLK_20210403_BEN.gpkg',
'44QLL_20210518_BEN.gpkg',
'44QMG_20210413_BEN.gpkg',
'44QMH_20210403_BEN.gpkg',
'44QMJ_20210403_BEN.gpkg',
'44QMK_20210403_BEN.gpkg',
'44QML_20210403_BEN.gpkg',
'44QNG_20210405_BEN.gpkg',
'44QNH_20210515_BEN.gpkg',
'44QNL_20210413_BEN.gpkg',
'44QPG_20210430_BEN.gpkg',
'44QPH_20210530_BEN.gpkg',
'44QPJ_20210405_BEN.gpkg',
'44QPK_20210420_BEN.gpkg',
'44QPL_20210405_BEN.gpkg',
'44QQJ_20210420_BEN.gpkg',
'44QQK_20210405_BEN.gpkg',
'44QQL_20210420_BEN.gpkg',
'44QRG_20210417_BEN.gpkg',
'44QRH_20210407_BEN.gpkg',
'44QRJ_20210407_BEN.gpkg',
'44QRK_20210522_BEN.gpkg',
]